In [144]:
from FRC_API import FRC_API_Interface
import pandas as pd
from typing import Any, TypedDict, NotRequired
import time
import numpy as np

In [21]:
import logging
logging.basicConfig(
    format="{levelname}: {message}", 
    style='{',
    level=logging.DEBUG,
)

First, we need to get all of our data. We'll cache it so we don't have to make hundreds of API calls every time we rerun the notebook. Eventually, we want pretty much every pick at an official event that is in the FRC API. I'll define a pick as an integer that is >=0 that represents the rank out of the available teams of the team picked for an alliance. To get this, we need to start by determining every event, then getting the alliance selection and team data for them. We then have a couple options for how to convert this data to the picks we are looking for. I'll start by gathering all of the data we need from the FRC API.

In [165]:
# Get all of the valid years
FRC_API = FRC_API_Interface()

min_season = 2006
current_season = 2024 # Should be changed to automatically update once the ancillary data is available on the FRC API
seasons = range(min_season, current_season + 1)

events_cache ='../cache/picking_probs/events.pkl'

class Event(TypedDict):
    season: int 
    code: str 
    type: str
    districtCode: str
    rankings: NotRequired[dict[int, int]] # team: rank
    alliances: NotRequired[list[list[int]]] # list[list[team]]

try:
    events_table: pd.DataFrame = pd.read_pickle(events_cache)
    logging.debug('Cache hit, checking dataframe format')

    if not all([key in events_table.columns for key in Event.__required_keys__]):
        raise Exception('Missing required columns')
    
    for key in Event.__optional_keys__:
        if key not in events_table.columns:
            events_table[key] = np.nan
            logging.debug(f'Added missing optional column {key}')

except Exception as e:
    logging.debug(e)
    logging.debug('Creating a new dataframe')
    events_table = pd.DataFrame(columns=Event.__annotations__)

events_data: list[Event] = []

for season in seasons:
    if season in events_table['season'].values:
        continue

    try:
        response = FRC_API.request(f'{season}/events')
        events: list[Any] = response['Events']

        for event in events:
            events_data.append({
                'season': season,
                'code': event['code'],
                'type': event['type'],
                'districtCode': event['districtCode'],
            })

    except Exception as e:
        logging.error(f'{season=}\n{e}')

    time.sleep(1)

events_table = pd.concat([events_table, pd.DataFrame(events_data)], ignore_index=True)

with pd.option_context(
    'display.max_rows', None,
    'display.max_columns', None,
    'display.width', 1000,
    ):
    logging.info(f'Events Table:\n{events_table}')

events_table.to_pickle(events_cache)

DEBUG: Cache hit, checking dataframe format


INFO: Events Table:
     season        code                            type districtCode rankings alliances
0      2006  ARCHIMEDES         ChampionshipSubdivision         None      NaN       NaN
1      2006          AZ                        Regional         None      NaN       NaN
2      2006          CA                        Regional         None      NaN       NaN
3      2006         CMP                    Championship         None      NaN       NaN
4      2006          CO                        Regional         None      NaN       NaN
5      2006          CT                        Regional         None      NaN       NaN
6      2006       CURIE         ChampionshipSubdivision         None      NaN       NaN
7      2006          DT                        Regional         None      NaN       NaN
8      2006          FL                        Regional         None      NaN       NaN
9      2006          GA                        Regional         None      NaN       NaN
10     2006 

In [158]:
# Create a subset of the event table for testing purposes
events_table = events_table[(events_table['season'] == 2024) & (events_table['districtCode'] == 'PNW')].reset_index(drop=True)
logging.info(f'Events Table:\n{events_table}')

INFO: Events Table:
  season   code                  type districtCode  alliances  rankings
0   2024  ORORE         DistrictEvent          PNW        NaN       NaN
1   2024  ORSAL         DistrictEvent          PNW        NaN       NaN
2   2024  ORWIL         DistrictEvent          PNW        NaN       NaN
3   2024  PNCMP  DistrictChampionship          PNW        NaN       NaN
4   2024  WAAHS         DistrictEvent          PNW        NaN       NaN
5   2024  WABON         DistrictEvent          PNW        NaN       NaN
6   2024  WASAM         DistrictEvent          PNW        NaN       NaN
7   2024  WASNO         DistrictEvent          PNW        NaN       NaN
8   2024  WAYAK         DistrictEvent          PNW        NaN       NaN
